# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

# Import

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.models.autoencoders import build_conv_autoencoder, build_dense_autoencoder
from debruiteur.models.gan import GAN
from debruiteur.noise.noise import *
from debruiteur.plots.plots import plot_model_loss
from debruiteur.preprocessing.preprocessor import *
from debruiteur.utils.utils import save_model, split_train_val_df

import os

# Configurations

In [2]:
noise_class_list = [
    GaussianNoise(mean=0, std=10),
    PoissonNoise(),
    UniformNoise(amplitude=100),
    SaltPepperNoise(freq=0.3),
    SquareMaskNoise(mask_shape=(10, 10), freq=0.1),
    SpeckleNoise(),
    AveragingBlurNoise(),
    GaussianBlurNoise(),
    MedianBlurNoise()
]

# Create Dataframe

In [3]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, "images"))
df_resized = make_resized_dataframe(df_original, img_shape=(64, 64, 1), resized_path=os.path.join(working_dir, "resized_images"))
df_noised = make_noised_dataframe(df_resized, noise_class_list, os.path.join(working_dir, "noised_images"))
df_train, df_val = split_train_val_df(df_noised)

# Models training

## Data generator

In [ ]:
train_gen = DataGenerator(df_train)
val_gen = DataGenerator(df_val)

## Convolutional autoencoder

In [ ]:
ae_model = build_conv_autoencoder()
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "conv_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## Dense autoencoder

In [ ]:
train_gen = DataGenerator(df_train, img_shape=(10000, ))
val_gen = DataGenerator(df_val, img_shape=(10000, ))

In [ ]:
ae_model = build_dense_autoencoder(loss="mse")
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "dense_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## GAN autoencoder

In [10]:
train_gen = DataGenerator(df_train, img_shape=(64, 64, 1), batch_size=8)
val_gen = DataGenerator(df_val, img_shape=(64, 64, 1), batch_size=8)

In [15]:
gan = GAN(img_shape=(64, 64, 1))

In [16]:
gan.generator.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 64, 64, 32)   2624        input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 64, 64, 32)   128         conv2d_20[0][0]                  
__________________________________________________________________________________________________
gen_conv1_relu (Activation)     (None, 64, 64, 32)   0           batch_normalization_16[0][0]     
____________________________________________________________________________________________

In [17]:
gan.discriminator.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 48)        816       
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 48)        192       
_________________________________________________________________
disc_conv1_relu (Activation) (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 16, 96)        73824     
_________________________________________________________________
batch_normalization_15 (Batc (None, 16, 16, 96)        384       
_________________________________________________________________
disco_conv2_relu (Activation (None, 16, 16, 96)        0   

In [19]:
train_hist, val_hist = gan.train(train_gen, val_gen, batch_size=8)

Epoch 1/10
229/229 [==============================] - 564s 2s/step
Validation for epoch 1
8/8 [==============================] - 0s 1ms/sample - loss: 2.8488
Train generator loss 0.08959231525659561
Train discriminator loss 0.26145556569099426
Validation generator loss 0.1570180058479309
Validation generator loss 2.0892606128345834
Epoch 2/10
  7/229 [..............................] - ETA: 9:12

KeyboardInterrupt: 